In [5]:
# 导入工具包
from PIL import Image
import numpy as np
import torchvision
from torch.utils.data import DataLoader
import torch

In [8]:
train_data = torchvision.datasets.MNIST(
    root = './mnist',
    # train = True 为 训练集，6w张照片；train = False 为 测试集，1w张照片
    train = True,
    download=True
    # download 在第一次加载数据集时需要指定为 True，后面就不需要了，即从./mnist 目录中加载
    # download = True
) # 通过此句，现 6W 中张照片均在 train_data 中，每一张照片都是 28 * 28 大小

# 构建可迭代的数据装载器，这里我们设置的 batch_size = 1，即一次循环，读取一张图片
train_loader = DataLoader(dataset=train_data, batch_size=1, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 192080865.44it/s]


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 49222142.96it/s]


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 44728310.68it/s]


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 1739297.80it/s]


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



In [9]:
def img_show(img):
    pil = Image.fromarray(np.uint(img))
    pil.show()

In [11]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

In [12]:
# 分批次训练，一批 64 个
BATCH_SIZE = 64
# 所有样本训练 3 次
EPOCHS = 3
# 学习率设置为 0.0006
LEARN_RATE = 6e-4

# 若当前 Pytorch 版本以及电脑支持GPU，则使用 GPU 训练，否则使用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [14]:
# 训练集数据加载
train_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor()
)
# 构建训练集的数据装载器，一次迭代有 BATCH_SIZE 张图片
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# 测试集数据加载
test_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=False,
    transform=torchvision.transforms.ToTensor()
)
# 构建测试集的数据加载器，一次迭代 1 张图片，我们一张一张的测试
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle = True)

In [15]:
"""
此处我们定义了一个 3 层的网络
隐藏层 1：40 个神经元
隐藏层 2：20 个神经元
输出层：10 个神经元
"""
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        # 隐藏层 1，使用 sigmoid 激活函数
        self.layer1 = nn.Sequential(
            nn.Linear(784, 40),
            nn.Sigmoid()
        )
        # 隐藏层 2，使用 sigmoid 激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(40, 20),
            nn.Sigmoid()
        )
        # 输出层
        self.layer_out = nn.Linear(20, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        self.out = self.layer_out(x)
        return self.out

In [16]:
# 实例化DNN，并将模型放在 GPU 训练
model = DNN().to(device)
# 同样，将损失函数放在 GPU
loss_fn = nn.MSELoss(reduction='mean').to(device)
#大数据常用Adam优化器，参数需要model的参数，以及学习率
optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE)

In [17]:
for epoch in range(EPOCHS):
    # 加载训练数据
    for step, data in enumerate(train_loader):
        x, y = data
        """
        因为此时的训练集即 x 大小为 （BATCH_SIZE, 1, 28, 28）
        因此这里需要一个形状转换为（BATCH_SIZE, 784）;
        y 中代表的是每张照片对应的数字，而我们输出的是 10 个神经元，
        即代表每个数字的概率
        因此这里将 y 也转换为该数字对应的 one-hot形式来表示
        """
        x = x.view(x.size(0), 784)
        yy = np.zeros((x.size(0), 10))
        for j in range(x.size(0)):
            yy[j][y[j].item()] = 1
        yy = torch.from_numpy(yy)
        yy = yy.float()
        x, yy = x.to(device), yy.to(device)

        # 调用模型预测
        output = model(x).to(device)
        # 计算损失值
        loss = loss_fn(output, yy)
        # 输出看一下损失变化
        print(f'EPOCH({epoch})   loss = {loss.item()}')
        # 每一次循环之前，将梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 梯度下降，更新参数
        optimizer.step()

EPOCH(0)   loss = 0.2225799262523651
EPOCH(0)   loss = 0.20510371029376984
EPOCH(0)   loss = 0.1990753710269928
EPOCH(0)   loss = 0.20555846393108368
EPOCH(0)   loss = 0.19732172787189484
EPOCH(0)   loss = 0.18653002381324768
EPOCH(0)   loss = 0.17645931243896484
EPOCH(0)   loss = 0.1719667613506317
EPOCH(0)   loss = 0.17414575815200806
EPOCH(0)   loss = 0.17915859818458557
EPOCH(0)   loss = 0.17049366235733032
EPOCH(0)   loss = 0.16227462887763977
EPOCH(0)   loss = 0.16656319797039032
EPOCH(0)   loss = 0.15101471543312073
EPOCH(0)   loss = 0.14999613165855408
EPOCH(0)   loss = 0.14728406071662903
EPOCH(0)   loss = 0.14207802712917328
EPOCH(0)   loss = 0.1471279412508011
EPOCH(0)   loss = 0.14282070100307465
EPOCH(0)   loss = 0.1293415129184723
EPOCH(0)   loss = 0.12704333662986755
EPOCH(0)   loss = 0.12669816613197327
EPOCH(0)   loss = 0.13012880086898804
EPOCH(0)   loss = 0.1286575049161911
EPOCH(0)   loss = 0.12279461324214935
EPOCH(0)   loss = 0.12215769290924072
EPOCH(0)   loss = 

In [18]:
sum = 0
# test：
for i, data in enumerate(test_loader):

    x, y = data
    # 这里 仅对 x 进行处理
    x = x.view(x.size(0), 784)
    x, y = x.to(device), y.to(device)
    res = model(x).to(device)
    # 得到 模型预测值
    r = torch.argmax(res)
    # 标签，即真实值
    l = y.item()
    sum += 1 if r == l else 0
    print(f'test({i})     DNN:{r} -- label:{l}')

print('accuracy：', sum / 10000)

串流輸出內容已截斷至最後 5000 行。
test(5001)     DNN:8 -- label:8
test(5002)     DNN:3 -- label:3
test(5003)     DNN:2 -- label:2
test(5004)     DNN:1 -- label:1
test(5005)     DNN:3 -- label:3
test(5006)     DNN:6 -- label:6
test(5007)     DNN:3 -- label:3
test(5008)     DNN:9 -- label:9
test(5009)     DNN:1 -- label:1
test(5010)     DNN:7 -- label:7
test(5011)     DNN:1 -- label:1
test(5012)     DNN:7 -- label:7
test(5013)     DNN:6 -- label:6
test(5014)     DNN:8 -- label:8
test(5015)     DNN:0 -- label:0
test(5016)     DNN:5 -- label:5
test(5017)     DNN:3 -- label:3
test(5018)     DNN:2 -- label:2
test(5019)     DNN:5 -- label:5
test(5020)     DNN:7 -- label:7
test(5021)     DNN:6 -- label:6
test(5022)     DNN:3 -- label:3
test(5023)     DNN:0 -- label:0
test(5024)     DNN:0 -- label:0
test(5025)     DNN:2 -- label:6
test(5026)     DNN:2 -- label:2
test(5027)     DNN:1 -- label:1
test(5028)     DNN:2 -- label:2
test(5029)     DNN:8 -- label:8
test(5030)     DNN:6 -- label:6
test(5031)     DNN:

In [19]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader

# 分批次训练，一批 64 个
BATCH_SIZE = 64
# 所有样本训练 3 次
EPOCHS = 3
# 学习率设置为 0.0006
LEARN_RATE = 6e-4

# 若当前 Pytorch 版本以及电脑支持GPU，则使用 GPU 训练，否则使用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 训练集数据加载
train_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=True,
    transform=torchvision.transforms.ToTensor()
)
# 构建训练集的数据装载器，一次迭代有 BATCH_SIZE 张图片
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# 测试集数据加载
test_data = torchvision.datasets.MNIST(
    root='./mnist',
    train=False,
    transform=torchvision.transforms.ToTensor()
)
# 构建测试集的数据加载器，一次迭代 1 张图片，我们一张一张的测试
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle = True)

"""
此处我们定义了一个 3 层的网络
隐藏层 1：40 个神经元
隐藏层 2：20 个神经元
输出层：10 个神经元
"""
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        # 隐藏层 1，使用 sigmoid 激活函数
        self.layer1 = nn.Sequential(
            nn.Linear(784, 40),
            nn.Sigmoid()
		)
        # 隐藏层 2，使用 sigmoid 激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(40, 20),
            nn.Sigmoid()
        )
        # 输出层
        self.layer_out = nn.Linear(20, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        self.out = self.layer_out(x)
        return self.out

# 实例化DNN，并将模型放在 GPU 训练
model = DNN().to(device)
# 同样，将损失函数放在 GPU
loss_fn = nn.MSELoss(reduction='mean').to(device)
#大数据常用Adam优化器，参数需要model的参数，以及学习率
optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE)

for epoch in range(EPOCHS):
    # 加载训练数据
    for step, data in enumerate(train_loader):
        x, y = data
        """
        因为此时的训练集即 x 大小为 （BATCH_SIZE, 1, 28, 28）
        因此这里需要一个形状转换为（BATCH_SIZE, 784）;

        y 中代表的是每张照片对应的数字，而我们输出的是 10 个神经元，
        即代表每个数字的概率
        因此这里将 y 也转换为该数字对应的 one-hot形式来表示
        """
        x = x.view(x.size(0), 784)
        yy = np.zeros((x.size(0), 10))
        for j in range(x.size(0)):
            yy[j][y[j].item()] = 1
        yy = torch.from_numpy(yy)
        yy = yy.float()
        x, yy = x.to(device), yy.to(device)

        # 调用模型预测
        output = model(x).to(device)
        # 计算损失值
        loss = loss_fn(output, yy)
        # 输出看一下损失变化
        print(f'EPOCH({epoch})   loss = {loss.item()}')
        # 每一次循环之前，将梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 梯度下降，更新参数
        optimizer.step()

sum = 0
# test：
for i, data in enumerate(test_loader):

    x, y = data
    # 这里 仅对 x 进行处理
    x = x.view(x.size(0), 784)
    x, y = x.to(device), y.to(device)
    res = model(x).to(device)
    # 得到 模型预测值
    r = torch.argmax(res)
    # 标签，即真实值
    l = y.item()
    sum += 1 if r == l else 0
    print(f'test({i})     DNN:{r} -- label:{l}')

print('accuracy：', sum / 10000)

串流輸出內容已截斷至最後 5000 行。
test(5001)     DNN:9 -- label:9
test(5002)     DNN:4 -- label:4
test(5003)     DNN:4 -- label:4
test(5004)     DNN:1 -- label:1
test(5005)     DNN:8 -- label:8
test(5006)     DNN:2 -- label:2
test(5007)     DNN:0 -- label:0
test(5008)     DNN:2 -- label:2
test(5009)     DNN:4 -- label:4
test(5010)     DNN:5 -- label:5
test(5011)     DNN:5 -- label:5
test(5012)     DNN:4 -- label:2
test(5013)     DNN:3 -- label:3
test(5014)     DNN:0 -- label:0
test(5015)     DNN:9 -- label:9
test(5016)     DNN:0 -- label:0
test(5017)     DNN:6 -- label:6
test(5018)     DNN:1 -- label:1
test(5019)     DNN:8 -- label:8
test(5020)     DNN:7 -- label:8
test(5021)     DNN:3 -- label:3
test(5022)     DNN:1 -- label:1
test(5023)     DNN:5 -- label:5
test(5024)     DNN:7 -- label:7
test(5025)     DNN:3 -- label:3
test(5026)     DNN:6 -- label:6
test(5027)     DNN:5 -- label:5
test(5028)     DNN:0 -- label:0
test(5029)     DNN:5 -- label:5
test(5030)     DNN:8 -- label:8
test(5031)     DNN: